# Used Cars Transform

In [1]:
import json
import random
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import psycopg2
from uszipcode import SearchEngine
import warnings

# Append the '../utils' directory to the sys.path for module imports
import sys
sys.path.append('./../utils')

# Ignore warnings
warnings.filterwarnings('ignore')

# Import custom database operations module
import db_operations

C:\Users\sampi\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


## Load the data

In [45]:
sql='''SELECT *
FROM used_cars
'''
cars_df = db_operations.run_query(sql)

Conexión exitosa!!


**Display the first few rows of the DataFrame**

In [46]:
cars_df.head()

,vin,back_legroom,bed,bed_height,bed_length,body_type,cabin,city,city_fuel_economy,combine_fuel_economy,...,transmission,transmission_display,trimid,trim_name,vehicle_damage_category,wheel_system,wheel_system_display,wheelbase,width,year
0,ZACNJABB5KPJ92081,35.1 in,NaN,NaN,NaN,SUV / Crossover,NaN,Bayamon,NaN,NaN,...,A,9-Speed Automatic Overdrive,t83804,Latitude FWD,NaN,FWD,Front-Wheel Drive,101.2 in,79.6 in,2019
1,SALCJ2FX1LH858117,38.1 in,NaN,NaN,NaN,SUV / Crossover,NaN,San Juan,NaN,NaN,...,A,9-Speed Automatic Overdrive,t86759,S AWD,NaN,AWD,All-Wheel Drive,107.9 in,85.6 in,2020
2,JF1VA2M67G9829723,35.4 in,NaN,NaN,NaN,Sedan,NaN,Guaynabo,17.0,NaN,...,M,6-Speed Manual,t58994,Base,NaN,AWD,All-Wheel Drive,104.3 in,78.9 in,2016
3,SALRR2RV0L2433391,37.6 in,NaN,NaN,NaN,SUV / Crossover,NaN,San Juan,NaN,NaN,...,A,8-Speed Automatic Overdrive,t86074,V6 HSE AWD,NaN,AWD,All-Wheel Drive,115 in,87.4 in,2020
4,SALCJ2FXXLH862327,38.1 in,NaN,NaN,NaN,SUV / Crossover,NaN,San Juan,NaN,NaN,...,A,9-Speed Automatic Overdrive,t86759,S AWD,NaN,AWD,All-Wheel Drive,107.9 in,85.6 in,2020


**Removing Columns**

In [49]:
columns_to_drop = [
    'bed', 'bed_height', 'bed_length', 'cabin', 'combine_fuel_economy',
    'is_certified', 'is_cpo', 'is_oemcpo', 'vehicle_damage_category',
    'engine_cylinders', 'exterior_color', 'franchise_make', 'wheel_system',
    'transmission_display', 'latitude', 'longitude', 'back_legroom',
    'front_legroom', 'power', 'has_accidents', 'salvage', 'theft_title',
    'fleet', 'engine_displacement', 'description', 'interior_color',
    'franchise_dealer', 'main_picture_url', 'major_options', 'iscab',
    'trimid', 'trim_name', 'city_fuel_economy', 'highway_fuel_economy',
    'length', 'height', 'width', 'torque', 'wheelbase', 'fuel_tank_volume',
    'savings_amount', 'sp_id', 'listing_id'
]

cars_df.drop(columns=columns_to_drop, inplace=True)

KeyError: "['bed', 'bed_height', 'bed_length', 'cabin', 'combine_fuel_economy', 'is_certified', 'is_cpo', 'is_oemcpo', 'vehicle_damage_category', 'engine_cylinders', 'exterior_color', 'franchise_make', 'wheel_system', 'transmission_display', 'latitude', 'longitude', 'back_legroom', 'front_legroom', 'power', 'has_accidents', 'salvage', 'theft_title', 'fleet', 'engine_displacement', 'description', 'interior_color', 'franchise_dealer', 'main_picture_url', 'major_options', 'iscab', 'trimid', 'trim_name', 'city_fuel_economy', 'highway_fuel_economy', 'length', 'height', 'width', 'torque', 'wheelbase', 'fuel_tank_volume', 'savings_amount', 'sp_id', 'listing_id'] not found in axis"

**Note:**
The following columns have been removed based on the results of the Exploratory Data Analysis (EDA). For a detailed explanation of the reasons behind these removals, please refer to the `used_cars_eda_3m.ipynbb notebook`. 

## Handling Outliers

____________________
**price**

In [48]:
cars_df.boxplot(column='price')

<Axes: >

In the boxplot, it is evident that there are numerous outliers, highlighting the need for their identification and subsequent removal to maintain data quality and integrity.

In [50]:
mean_price = cars_df['price'].mean()
std_price = cars_df['price'].std()
threshold = 2 * std_price
outliers = cars_df[(cars_df['price'] < (mean_price - threshold)) | (cars_df['price'] > (mean_price + threshold))]
new_cars_df = cars_df[~cars_df['price'].isin(outliers['price'])]

In [51]:
new_cars_df['price'] = new_cars_df['price'].astype(int)
new_cars_df['price'].sample(5)

45384    22995
90462    23998
65161    19900
94816    36127
20300    21545
Name: price, dtype: int32

In [52]:
outliers.shape

(2859, 23)

After identifying and eliminating 2,859 outliers in the `price` variable using a threshold of 2 times the standard deviation, the format of the `price` column was changed to an integer.

____________________
**mileage**

In [53]:
mean_price = new_cars_df['mileage'].mean()
std_price = new_cars_df['mileage'].std()
threshold = 2 * std_price
outliers = new_cars_df[(new_cars_df['mileage'] < (mean_price - threshold)) | (new_cars_df['mileage'] > (mean_price + threshold))]
new_cars_df = new_cars_df[~new_cars_df['mileage'].isin(outliers['mileage'])]
new_cars_df['mileage'].fillna(new_cars_df['mileage'].mean(), inplace=True)

In [54]:
new_cars_df['mileage'] = new_cars_df['mileage'].astype(int)
new_cars_df['mileage'].sample(5)

68860     29199
72727     25010
43031    115687
12431         3
14139         0
Name: mileage, dtype: int32

In [55]:
outliers.shape

(5892, 23)

Similarly, in the `mileage` column, 5892 outliers were identified and removed using the same standard deviation method. Additionally, to handle missing values, NaN values in the `mileage` column were filled with the column's mean, and the `mileage` format was changed to an integer.

## Missing Values: Removing Nulls

____________________
**body_type**

In [56]:
new_cars_df['body_type'].value_counts()

body_type
SUV / Crossover    48694
Sedan              24926
Pickup Truck        7906
Hatchback           2365
Van                 1830
Coupe               1739
Minivan             1525
Wagon               1237
Convertible          700
NaN                  329
Name: count, dtype: int64

In [57]:
new_cars_df = new_cars_df[new_cars_df['body_type'] != 'NaN']
new_cars_df['body_type'].unique()

array(['SUV / Crossover', 'Sedan', 'Coupe', 'Hatchback', 'Pickup Truck',
       'Wagon', 'Minivan', 'Convertible', 'Van'], dtype=object)

Given the small number of missing values in the `body_type`column, it was decided to simply remove them.

____________________
**fuel_type**

In [58]:
new_cars_df = new_cars_df[new_cars_df['fuel_type'] != 'NaN']
new_cars_df['fuel_type'].unique()

array(['Gasoline', 'Diesel', 'Biodiesel', 'Flex Fuel Vehicle', 'Electric',
       'Hybrid', 'Compressed Natural Gas'], dtype=object)

The decision to remove the null values in the  `fuel_type` column was made because this information was essential for populating the `engine_type` column. Without knowledge of the fuel type, it wouldn't provide significant relevance.

## Data Transformations

____________________
**engine_type**

In [59]:
new_cars_df['engine_type'] = new_cars_df['engine_type'].str.split(' ').str[0]

In [60]:
nan_engine_type = cars_df[cars_df['engine_type'] == 'NaN']
nan_engine_type[['engine_type','fuel_type']].value_counts()

engine_type  fuel_type
NaN          NaN          2459
             Electric      457
             Diesel         75
             Gasoline        7
Name: count, dtype: int64

In [61]:
new_cars_df.loc[(new_cars_df['engine_type']== 'NaN') & (new_cars_df['fuel_type'] == "Electric"), 'engine_type'] = "Electric"
new_cars_df.loc[(new_cars_df['engine_type']== 'NaN') & (new_cars_df['fuel_type'] == "Diesel"), 'engine_type'] = "Diesel"
new_cars_df.loc[(new_cars_df['engine_type']== 'NaN') & (new_cars_df['fuel_type'] == "Gasoline"), 'engine_type'] = "Gasoline"
new_cars_df['engine_type'].unique()

array(['I4', 'H4', 'V6', 'I6', 'V8', 'Electric', 'I3', 'I5', 'R2',
       'Diesel', 'W12', 'H6', 'I2', 'V10', 'Gasoline', 'V12'],
      dtype=object)

Given the strong relationship between 'engine_type' and 'fuel_type' (e.g., 'V8 Diesel'), it was decided to retain only the 'engine_type' information and remove the 'fuel_type.' Where 'engine_type' was missing (NaN), it was assigned the value from 'fuel_type,' considering that many of these missing values corresponded to electric, diesel, or gasoline cars.

_______________
**transmission**

In [62]:
new_cars_df['transmission'].value_counts()

transmission
A              73328
CVT            13116
M               1392
NaN              824
Dual Clutch      318
Name: count, dtype: int64

In [63]:
new_cars_df['transmission'].replace('NaN', np.nan, inplace=True)
new_cars_df['transmission'].fillna('A', inplace=True)
new_cars_df['transmission'] = new_cars_df['transmission'].replace({'A': 'Automatic', 'CVT': 'Continuously Variable Transmission', 'M': 'Manual'})
new_cars_df['transmission'].unique()

array(['Automatic', 'Manual', 'Continuously Variable Transmission',
       'Dual Clutch'], dtype=object)

In general, the transformations in the  `transmission` column involved addressing missing values by replacing them with the most common category, 'Automatic' (represented as 'A'). Additionally, category labels were updated to provide clearer and more informative descriptions.

_______________
**listing_color**

In [64]:
new_cars_df['listing_color'] = new_cars_df['listing_color'].str.capitalize()
new_cars_df['listing_color'].unique()

array(['Yellow', 'Black', 'Unknown', 'Gray', 'Silver', 'White', 'Red',
       'Blue', 'Orange', 'Green', 'Brown', 'Teal', 'Purple', 'Gold',
       'Pink'], dtype=object)

The format of the  `listing_color` column was modified to capitalize the first letter of each word, providing a more standardized and visually appealing format.

_______________
**horsepower**

In [65]:
new_cars_df['horsepower'] = new_cars_df['horsepower'].apply(lambda x: random.uniform(180, 200) if pd.isna(x) else x)
new_cars_df['horsepower'] = new_cars_df['horsepower'].astype(int)
new_cars_df['horsepower'].sample(5)

59040    310
66531    138
43225    270
86808    190
39930    175
Name: horsepower, dtype: int32

To address missing values in the `horsepower`column, a decision was made to randomly assign values between 180 and 200. This range was chosen to reflect the average horsepower of a new car, which typically falls within this range. 

_____________________
**maximum_seating**

In [66]:
result = new_cars_df.groupby('body_type')['maximum_seating'].agg(lambda x: x.mode().iloc[0])
result

body_type
Convertible        4 seats
Coupe              4 seats
Hatchback          5 seats
Minivan            8 seats
Pickup Truck       6 seats
SUV / Crossover    5 seats
Sedan              5 seats
Van                2 seats
Wagon              5 seats
Name: maximum_seating, dtype: object

In [67]:
filtered_df = new_cars_df[(new_cars_df['maximum_seating'] == 'NaN') | (new_cars_df['maximum_seating'] == '--')]
body_types_with_nan = filtered_df['body_type'].unique()
body_types_with_nan

array(['SUV / Crossover', 'Sedan', 'Pickup Truck', 'Hatchback', 'Wagon',
       'Coupe', 'Van', 'Minivan', 'Convertible'], dtype=object)

In [68]:
new_cars_df['maximum_seating'].replace(["--", 'NaN'], np.nan, inplace=True)

In [71]:
default_seating_values = {
    'Convertible': '4 seats',
    'Coupe': '4 seats',
    'Hatchback': '5 seats',
    'Minivan': '8 seats',
    'Pickup Truck': '6 seats',
    'SUV / Crossover': '5 seats',
    'Sedan': '5 seats',
    'Van': '2 seats',
    'Wagon': '5 seats'
}
new_cars_df['maximum_seating'] = new_cars_df.apply(lambda row: default_seating_values.get(row['body_type'], row['maximum_seating']) if pd.isna(row['maximum_seating']) else row['maximum_seating'], axis=1)
new_cars_df['maximum_seating'] = new_cars_df['maximum_seating'].str.replace(" seats", "").astype(int)

In [72]:
new_cars_df['maximum_seating'].unique()

array([ 5,  7,  4,  8,  6,  2,  9,  3, 15, 12, 10])

To address missing values in the `maximum_seating` column, the most common value (mode) for 'maximum_seating' was assigned based on the `body_type` group. Additionally, ' seats' was removed from the string, and the values were converted to integers to ensure consistent data format.

___________________
**seller_rating**

In [73]:
new_cars_df['seller_rating'] = new_cars_df['seller_rating'].apply(np.ceil)
new_cars_df['seller_rating'].fillna(0, inplace=True)
new_cars_df['seller_rating'] = new_cars_df['seller_rating'].astype(int)
new_cars_df['seller_rating'].unique()

array([3, 0, 4, 5, 1, 2])

To simplify the `seller_rating` column, only the integer part of the rating was retained. Any missing values were assigned a reputation score of 0. The entire `seller_rating` column was subsequently converted to integers.

____________________
**owner_count**

In [74]:
new_cars_df['owner_count'].fillna(-1, inplace=True)
new_cars_df['owner_count']= new_cars_df['owner_count'].astype(int)
new_cars_df['owner_count'].unique()

array([-1,  3,  2,  1,  5,  4,  6,  7,  9,  8, 10, 12, 15])

To address missing values in the `owner_count` column, an identifier of -1 was used to indicate cases where the number of owners is unknown. This approach was chosen to retain the data while acknowledging the absence of owner count information.

__________________
**frame_damaged**

In [75]:
new_cars_df['frame_damaged'].replace(['NaN'], np.nan, inplace=True)
new_cars_df['frame_damaged'].fillna('false', inplace=True)
new_cars_df['frame_damaged'].replace('false',0, inplace=True)
new_cars_df['frame_damaged'].replace('true',1, inplace=True)
new_cars_df['frame_damaged'].unique()

array([0, 1], dtype=int64)

To handle missing values in the `frame_damaged` column, any null values were replaced with 0 indicating that no frame damage was reported. After this transformation, the `frame_damaged` column now only contains either 1 or 0 values.

__________________
**is_new**

In [76]:
new_cars_df['is_new'].replace(False,0, inplace=True)
new_cars_df['is_new'].replace(True,1, inplace=True)
new_cars_df['is_new'].unique()

array([1, 0], dtype=int64)

For convenience, the `is_new` column was transformed by replacing True with 1 and False with 0.


_______________
**dealer_zip**

In [31]:
def get_state(zip_code):
    engine = SearchEngine()
    try:
        zipcode = engine.by_zipcode(zip_code)
        return zipcode.state_long
    except:
        return np.nan

# Apply the get_state function to 'dealer_zip' column
new_cars_df['dealer_state'] = new_cars_df['dealer_zip'].apply(lambda x: get_state(x))

In [32]:
new_cars_df['dealer_state'].unique()

array(['Puerto Rico', 'New York', 'Connecticut', 'New Jersey', 'Michigan',
       'Massachusetts', 'Illinois', 'Ohio', 'Pennsylvania', 'Kentucky',
       'Rhode Island', 'Indiana', 'Texas', 'Nebraska', 'New Hampshire',
       'Vermont', 'Maine'], dtype=object)

A new column, `dealer_state` was created to hold the state information obtained from the `dealer_zip` column, providing additional location details for the dataset. This transformation was carried out to enhance the dataset with geographic information.

**Data Transformation Summary**

In [33]:
new_cars_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 88977 entries, 0 to 100000
Data columns (total 24 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   vin                   88977 non-null  object        
 1   body_type             88977 non-null  object        
 2   city                  88977 non-null  object        
 3   daysonmarket          88977 non-null  int64         
 4   dealer_zip            88977 non-null  int64         
 5   engine_type           88977 non-null  object        
 6   frame_damaged         88977 non-null  int64         
 7   fuel_type             88977 non-null  object        
 8   horsepower            88977 non-null  int32         
 9   is_new                88977 non-null  int64         
 10  listed_date           88977 non-null  datetime64[ns]
 11  listing_color         88977 non-null  object        
 12  make_name             88977 non-null  object        
 13  maximum_seating     

In [77]:
new_cars_df.to_csv('data.csv')

After completing all the necessary transformations, the dataset no longer contains any missing or null values. The dataset is now clean and ready for further analysis and is prepared for storage.

## Data Warehouse Design

In [34]:
new_cars_df_copy = new_cars_df.copy()

**Date Dimension**

In [35]:
date_df = new_cars_df[['listed_date']].copy()
date_df = date_df.drop_duplicates().reset_index(drop=True)
date_df['date_key'] = date_df['listed_date'].dt.strftime('%Y%m%d')
date_df['year'] = date_df['listed_date'].dt.year
date_df['short_month'] = date_df['listed_date'].dt.strftime('%b')
date_df['day_of_week'] = date_df['listed_date'].dt.strftime('%A')
date_df.rename(columns={'listed_date': 'date'}, inplace=True)
date_df = date_df[['date_key', 'date', 'short_month', 'day_of_week','year']]
date_df.head()

,date_key,date,short_month,day_of_week,year
0,20190406,2019-04-06,Apr,Saturday,2019
1,20200215,2020-02-15,Feb,Saturday,2020
2,20170425,2017-04-25,Apr,Tuesday,2017
3,20200226,2020-02-26,Feb,Wednesday,2020
4,20200425,2020-04-25,Apr,Saturday,2020


In [36]:
date_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 805 entries, 0 to 804
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   date_key     805 non-null    object        
 1   date         805 non-null    datetime64[ns]
 2   short_month  805 non-null    object        
 3   day_of_week  805 non-null    object        
 4   year         805 non-null    int32         
dtypes: datetime64[ns](1), int32(1), object(3)
memory usage: 28.4+ KB


In [37]:
new_cars_df_copy['date_key'] = new_cars_df_copy['listed_date'].dt.strftime('%Y%m%d')
new_cars_df_copy['date_key'].nunique()

805

The Date Dimension is designed to capture and organize information related to dates. It plays a critical role in time-based analysis and reporting.

**Model Dimension**

In [38]:
model_df = new_cars_df[['make_name', 'model_name', 'body_type']].copy()
model_df = model_df.drop_duplicates().reset_index(drop=True)
model_df['full_model_name'] = model_df['make_name'] + ' ' + model_df['model_name']
model_df['model_key'] = model_df.index
model_df = model_df[['model_key', 'make_name', 'model_name', 'full_model_name','body_type']]
model_df.head()

,model_key,make_name,model_name,full_model_name,body_type
0,0,Jeep,Renegade,Jeep Renegade,SUV / Crossover
1,1,Land Rover,Discovery Sport,Land Rover Discovery Sport,SUV / Crossover
2,2,Subaru,WRX STI,Subaru WRX STI,Sedan
3,3,Land Rover,Discovery,Land Rover Discovery,SUV / Crossover
4,4,Land Rover,Range Rover Velar,Land Rover Range Rover Velar,SUV / Crossover


In [39]:
model_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 807 entries, 0 to 806
Data columns (total 5 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   model_key        807 non-null    int64 
 1   make_name        807 non-null    object
 2   model_name       807 non-null    object
 3   full_model_name  807 non-null    object
 4   body_type        807 non-null    object
dtypes: int64(1), object(4)
memory usage: 31.7+ KB


In [40]:
new_cars_df_copy = new_cars_df_copy.merge(model_df, on=['make_name', 'model_name', 'body_type'], how='left')
new_cars_df_copy['model_key'].nunique()

807

The Model Dimension contains detailed information about vehicle models and their characteristics, which is crucial for analyzing and reporting on vehicle-related data.

**Location Dimension**

In [43]:
location_df = new_cars_df[['city', 'dealer_state']].copy()
location_df = location_df.drop_duplicates().reset_index(drop=True)
location_df['location_key'] = location_df.index
location_df['city_state'] = location_df['city'] + ', ' + location_df['dealer_state']
location_df.rename(columns={'dealer_state': 'state'}, inplace=True)
location_df = location_df[['location_key', 'city', 'state', 'city_state']]
location_df.head()

,location_key,city,state,city_state
0,0,Bayamon,Puerto Rico,"Bayamon, Puerto Rico"
1,1,San Juan,Puerto Rico,"San Juan, Puerto Rico"
2,2,Guaynabo,Puerto Rico,"Guaynabo, Puerto Rico"
3,3,Woodbury,New York,"Woodbury, New York"
4,4,Bohemia,New York,"Bohemia, New York"


In [44]:
location_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 439 entries, 0 to 438
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   location_key  439 non-null    int64 
 1   city          439 non-null    object
 2   state         439 non-null    object
 3   city_state    439 non-null    object
dtypes: int64(1), object(3)
memory usage: 13.8+ KB


In [45]:
new_cars_df_copy = new_cars_df_copy.merge(location_df, left_on=['dealer_state','city'],right_on=['state','city'], how='left')
new_cars_df_copy['location_key'].nunique()

439

The Location Dimension is created to capture and organize data related to geographic locations, providing insights into regional variations within the dataset.

**Dealer Dimension**

In [46]:
dealer_df = new_cars_df[['seller_rating', 'sp_name']].copy()
dealer_df = dealer_df.drop_duplicates().reset_index(drop=True)
dealer_df = dealer_df.groupby('sp_name').agg({'seller_rating': 'max'}).reset_index()
dealer_df['dealer_key'] = dealer_df.index
dealer_df = dealer_df.rename(columns={
    'dealer_key': 'dealer_key',
    'seller_rating': 'reputation_score',
    'sp_name': 'dealer_name',
})
dealer_df= dealer_df[['dealer_key','dealer_name','reputation_score']]
dealer_df.head()

,dealer_key,dealer_name,reputation_score
0,0,103 Auto Sales,5
1,1,112 Auto Plaza,5
2,2,1st Priority Autos,5
3,3,21st Century Auto Group,3
4,4,26 Motors,4


In [47]:
dealer_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 850 entries, 0 to 849
Data columns (total 3 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   dealer_key        850 non-null    int64 
 1   dealer_name       850 non-null    object
 2   reputation_score  850 non-null    int32 
dtypes: int32(1), int64(1), object(1)
memory usage: 16.7+ KB


In [48]:
new_cars_df_copy = new_cars_df_copy.merge(dealer_df, left_on=['sp_name'],right_on=['dealer_name'], how='left')
new_cars_df_copy['dealer_key'].nunique()

850

The Dealer Dimension is designed to aggregate essential data pertaining to dealers, emphasizing their reputation scores and names. 

**Fact Table**

In [49]:
fact_table = new_cars_df_copy

In [51]:
columns_to_exclude = [
    'reputation_score',
    'dealer_name',
    'city_state',
    'state',
    'full_model_name',
    'dealer_state',
    'sp_name',
    'seller_rating',
    'model_name',
    'listed_date',
    'city',
    'dealer_zip',
    'body_type',
    'make_name'   
]

fact_table = fact_table.drop(columns=columns_to_exclude)

In [52]:
fact_table = fact_table.rename(columns={
    'vin': 'vehicle_vin',
    'daysonmarket': 'days_on_market',
    'engine_type': 'engine',
    'frame_damaged': 'has_accidents',  # Rename 'frame_damaged' to 'has_accidents'
    'listing_color': 'color',  # Rename 'listing_color' to 'color'
    'maximum_seating': 'seating_capacity',
    'transmission': 'transmission',
    'wheel_system_display': 'drivetrain',
    'year': 'vehicle_year',
})

In [53]:
fact_table.head()

,vehicle_vin,days_on_market,engine,has_accidents,fuel_type,horsepower,is_new,color,seating_capacity,mileage,owner_count,price,transmission,drivetrain,vehicle_year,date_key,model_key,location_key,dealer_key
0,ZACNJABB5KPJ92081,522,I4,0,Gasoline,177,1,Yellow,5,7,-1,23141,Automatic,Front-Wheel Drive,2019,20190406,0,0,311
1,SALCJ2FX1LH858117,207,I4,0,Gasoline,246,1,Black,7,8,-1,46500,Automatic,All-Wheel Drive,2020,20200215,1,1,453
2,JF1VA2M67G9829723,1233,H4,0,Gasoline,305,0,Unknown,5,23945,3,46995,Manual,All-Wheel Drive,2016,20170425,2,2,297
3,SALRR2RV0L2433391,196,V6,0,Gasoline,340,1,Gray,7,11,-1,67430,Automatic,All-Wheel Drive,2020,20200226,3,1,453
4,SALCJ2FXXLH862327,137,I4,0,Gasoline,246,1,Black,7,7,-1,48880,Automatic,All-Wheel Drive,2020,20200425,1,1,453


In [54]:
fact_table.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 88977 entries, 0 to 88976
Data columns (total 19 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   vehicle_vin       88977 non-null  object
 1   days_on_market    88977 non-null  int64 
 2   engine            88977 non-null  object
 3   has_accidents     88977 non-null  int64 
 4   fuel_type         88977 non-null  object
 5   horsepower        88977 non-null  int32 
 6   is_new            88977 non-null  int64 
 7   color             88977 non-null  object
 8   seating_capacity  88977 non-null  int32 
 9   mileage           88977 non-null  int32 
 10  owner_count       88977 non-null  int32 
 11  price             88977 non-null  int32 
 12  transmission      88977 non-null  object
 13  drivetrain        88977 non-null  object
 14  vehicle_year      88977 non-null  int64 
 15  date_key          88977 non-null  object
 16  model_key         88977 non-null  int64 
 17  location_key

The fact table serves as the central repository for key information on used cars, including their attributes, pricing, and other relevant data. This table is essential for conducting in-depth analysis and generating valuable insights from the dataset.

## Data Warehouse Load

In [55]:
db_operations.create_data_warehouse()

Conexión exitosa!!
Data Warehouse creado exitosamente


In [56]:
db_operations.insert_data_warehouse(date_df,'date_dim')

Conexión exitosa!!
Los datos se han cargado en: date_dim


In [57]:
db_operations.insert_data_warehouse(model_df,'model_dim')

Conexión exitosa!!
Los datos se han cargado en: model_dim


In [58]:
db_operations.insert_data_warehouse(location_df,'location_dim')

Conexión exitosa!!
Los datos se han cargado en: location_dim


In [59]:
db_operations.insert_data_warehouse(dealer_df,'dealer_dim')

Conexión exitosa!!
Los datos se han cargado en: dealer_dim


In [60]:
db_operations.insert_data_warehouse(fact_table,'used_cars_facts')

Conexión exitosa!!
Los datos se han cargado en: used_cars_facts
